In [1]:
import sys
print(sys.version)

3.10.4 (v3.10.4:9d38120e33, Mar 23 2022, 17:29:05) [Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import xgboost

In [3]:
X,y = shap.datasets.boston()
X.shape



Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

(506, 13)

In [4]:
model = sklearn.ensemble.RandomForestRegressor(n_estimators=100, max_depth=6)
model.fit(X, y)



RandomForestRegressor(max_depth=6)

In [5]:
# bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [6]:
# start = time.time()
# shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
# print(time.time() - start)


In [7]:
from shap.explainers.pytree import *
# x = np.ones(X.shape[1])
# TreeExplainer(model).shap_values(x)

In [8]:
from shap import TreeExplainer as TreeCext
start = time.time()
t = TreeCext(model).shap_values(X, banz=False)
print(time.time() - start)
t

ile drzew cext? 100
using treeshap and cext
banz = False
here37
using dense_Tree_pred
0.13974905014038086


array([[ 3.31504158e-01,  2.98987879e-03,  1.03616071e-01, ...,
         2.55613088e-01, -6.02482768e-02,  4.67681086e+00],
       [ 3.76612809e-01, -1.22931327e-03, -1.30528718e-02, ...,
         1.58777187e-01, -1.53768033e-02,  1.51930890e+00],
       [ 2.43807382e-01, -1.33371082e-02, -1.51119155e-01, ...,
         1.57033988e-01, -1.64608587e-02,  4.70779476e+00],
       ...,
       [ 3.04057444e-01, -2.74349807e-03, -1.44460165e-01, ...,
        -1.32819642e+00, -9.59289702e-02,  2.47183095e+00],
       [ 3.76060067e-01, -6.43719362e-03, -5.17213856e-02, ...,
        -8.78581017e-01,  3.08521511e-02,  3.68103616e+00],
       [ 3.19224779e-01, -7.18953301e-04, -7.78787901e-03, ...,
        -6.61786938e-01, -3.10558787e-03,  1.28015542e+00]])

In [9]:
start = time.time()
t = TreeCext(model).shap_values(X, banz=True)
print(time.time() - start)
t # TODO BLAD - dodają się zamiast zaczynac od 0 dla kazdej probki danych

ile drzew cext? 100
using treeshap and cext
banz = True
using cext banz
0.09666705131530762


array([[1.48681839e+00, 1.60008869e-03, 2.02454562e-01, ...,
        4.28819234e-01, 8.79015799e-02, 1.96834086e+01],
       [2.97363678e+00, 3.20017739e-03, 4.04909124e-01, ...,
        8.57638468e-01, 1.75803160e-01, 3.93668172e+01],
       [4.46045517e+00, 4.80026608e-03, 6.07363685e-01, ...,
        1.28645770e+00, 2.63704740e-01, 5.90502257e+01],
       ...,
       [7.49356469e+02, 8.06444702e-01, 1.02037099e+02, ...,
        2.16124894e+02, 4.43023963e+01, 9.92043792e+03],
       [7.50843287e+02, 8.08044790e-01, 1.02239554e+02, ...,
        2.16553713e+02, 4.43902978e+01, 9.94012133e+03],
       [7.52330106e+02, 8.09644879e-01, 1.02442008e+02, ...,
        2.16982532e+02, 4.44781994e+01, 9.95980474e+03]])

In [10]:
start = time.time()
ex = TreeExplainer(model)
print(time.time() - start)
start = time.time()
print(ex.shap_values(X))
print(time.time() - start)

0.023111820220947266
oryginalna implemenacja shap_values
other model type
using slow python treeshap
[[ 3.31504158e-01  2.98987879e-03  1.03616071e-01 ... -6.02482768e-02
   4.67681086e+00  2.25744308e+01]
 [ 3.76612809e-01 -1.22931327e-03 -1.30528718e-02 ... -1.53768033e-02
   1.51930890e+00  2.25744308e+01]
 [ 2.43807382e-01 -1.33371082e-02 -1.51119155e-01 ... -1.64608587e-02
   4.70779476e+00  2.25744308e+01]
 ...
 [ 3.04494810e-01 -2.74349807e-03 -1.44460165e-01 ... -9.59289702e-02
   2.47183095e+00  2.25744308e+01]
 [ 3.76078190e-01 -6.43719362e-03 -5.17213856e-02 ...  3.08521511e-02
   3.68103616e+00  2.25744308e+01]
 [ 3.19242903e-01 -7.18953301e-04 -7.78787901e-03 ... -3.10558787e-03
   1.28015542e+00  2.25744308e+01]]
61.58555197715759


In [11]:
start = time.time()
res = ex.banz_values(X)
print(time.time() - start)
res

nasza implementacja banz values
other model type
wlasciwe obliczenia w sekundzie: 0.0013759136199951172
dwa wymiary
37.885870933532715


[[0.34192623294640223,
  0.0012633346491450553,
  0.10633551767970921,
  -0.0006645339075481569,
  -0.018139773603082082,
  -0.9399705744590408,
  0.018822925919070213,
  -0.19364161244299483,
  -0.05020506982833526,
  -0.07419483146639597,
  0.262389095189707,
  -0.054147917185250805,
  4.742012835694186],
 [0.3897829977478275,
  -0.0015205913272214558,
  -0.01500058815507303,
  -0.00044260556365665395,
  0.3159418748724648,
  -2.355320789201762,
  0.033556702915166416,
  -0.2220886970602852,
  0.006820243724806731,
  0.065342441001436,
  0.1659308209138249,
  -0.015210744429937235,
  1.5476453372667465],
 [0.20720707086399795,
  -0.0162110753090743,
  -0.14959209597919937,
  -0.007977868986426407,
  0.35233808115965054,
  6.230868991224389,
  0.08344763730123493,
  -0.2559105088658432,
  -0.021662141207887595,
  0.2546381002220034,
  0.1379457609495782,
  -0.007389643348730046,
  4.6075307300771415],
 [0.19744165972682162,
  -0.0015908194904514117,
  0.10554614225060018,
  -0.0079009

In [12]:
start = time.time()
res = ex.banz_values(X, jit=True)
print(time.time() - start)
res

nasza implementacja banz values
other model type
using numba py banz
wlasciwe obliczenia w sekundzie: 0.001338958740234375
dwa wymiary
37.838075160980225


[[0.34192623294640223,
  0.0012633346491450553,
  0.10633551767970921,
  -0.0006645339075481569,
  -0.018139773603082082,
  -0.9399705744590408,
  0.018822925919070213,
  -0.19364161244299483,
  -0.05020506982833526,
  -0.07419483146639597,
  0.262389095189707,
  -0.054147917185250805,
  4.742012835694186],
 [0.3897829977478275,
  -0.0015205913272214558,
  -0.01500058815507303,
  -0.00044260556365665395,
  0.3159418748724648,
  -2.355320789201762,
  0.033556702915166416,
  -0.2220886970602852,
  0.006820243724806731,
  0.065342441001436,
  0.1659308209138249,
  -0.015210744429937235,
  1.5476453372667465],
 [0.20720707086399795,
  -0.0162110753090743,
  -0.14959209597919937,
  -0.007977868986426407,
  0.35233808115965054,
  6.230868991224389,
  0.08344763730123493,
  -0.2559105088658432,
  -0.021662141207887595,
  0.2546381002220034,
  0.1379457609495782,
  -0.007389643348730046,
  4.6075307300771415],
 [0.19744165972682162,
  -0.0015908194904514117,
  0.10554614225060018,
  -0.0079009